In [1]:
import tkinter as tk
from tkinter import messagebox
import cv2
import os
from PIL import Image
import numpy as np
import mysql.connector

In [2]:
# Function definitions
def train_classifier():
    data_dir = "C:/Users/anshu/OneDrive/Desktop/CodSoft/Face Detection & Recognition/data"
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    faces = []
    ids = []
    
    for image in path:
        img = Image.open(image).convert('L')  # Convert image to grayscale
        imageNp = np.array(img, 'uint8')
        
        id = int(os.path.split(image)[1].split(".")[1])
        faces.append(imageNp)
        ids.append(id)
    
    ids = np.array(ids)
    
    if not hasattr(cv2, 'face'):
        print("Error: The 'face' module is not available. Please install opencv-contrib-python.")
        exit()

    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces, ids)
    clf.write("classifier.xml")
    messagebox.showinfo('Result','Training dataset completed !!!')

def detect_face():
    def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
        gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        features = classifier.detectMultiScale(gray_image, scaleFactor, minNeighbors)
    
        coords = []
    
        for (x, y, w, h) in features:
            cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)
            id, pred = clf.predict(gray_image[y:y+h, x:x+w])
            confidence = int(100 * (1 - pred / 300))

            mydb = mysql.connector.connect(
                host="localhost",
                user="root",
                passwd="",
                database="Authorized_user"
            )
            mycursor = mydb.cursor()
            mycursor.execute("select name from my_table where id=" + str(id))
            s = mycursor.fetchone()
            s = '' + ''.join(s)

            if confidence > 72:
                cv2.putText(img, s, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
            else:
                cv2.putText(img, "UNKNOWN", (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 1, cv2.LINE_AA)
            
            coords = [x, y, h, w]
        return coords

    def recognize(img, clf, faceCascade):
        coords = draw_boundary(img, faceCascade, 1.1, 10, (255, 255, 255), "Face", clf)
        return img

    faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.read("classifier.xml")
    
    video_capture = cv2.VideoCapture(0)
    
    while True:
        ret, img = video_capture.read()
        img = recognize(img, clf, faceCascade)
        cv2.imshow("face detection", img)
    
        if cv2.waitKey(1) == 13:
            break
    
    video_capture.release()
    cv2.destroyAllWindows()

def generate_dataset():
    if t1.get() == "" or t2.get() == "" or t3.get() == "" or t4.get() == "" or t5.get() == "":
        messagebox.showinfo('Result', 'Please provide complete details of the user')
    else:
        mydb = mysql.connector.connect(
            host="localhost",
            user="root",
            passwd="",
            database="Authorized_user"
        )
        mycursor = mydb.cursor()
        mycursor.execute("SELECT * from my_table")
        myresult = mycursor.fetchall()
        id = 1
        for x in myresult:
            id += 1
        
        # Modify the SQL query to include email and contact
        sql = "insert into my_table(id, Name, Age, Address, Email, Contact) values(%s, %s, %s, %s, %s, %s)"
        val = (id, t1.get(), t2.get(), t3.get(), t4.get(), t5.get())  # Include email and contact
        mycursor.execute(sql, val)
        mydb.commit()
        
        face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
        
        def face_cropped(img):
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_classifier.detectMultiScale(gray, 1.3, 5)
            
            if len(faces) == 0:
                return None
            
            for (x, y, w, h) in faces:
                cropped_face = img[y:y+h, x:x+w]
                return cropped_face
            
        cap = cv2.VideoCapture(0)
        img_id = 0
    
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            cropped_face = face_cropped(frame)
            if cropped_face is not None:
                img_id += 1
                face = cv2.resize(cropped_face, (200, 200))
                face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
                
                file_name_path = f"data/user.{id}.{img_id}.jpg"
                cv2.imwrite(file_name_path, face)
                
                cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
                cv2.imshow("Cropped face", face)
            
            if cv2.waitKey(1) == 13 or img_id == 200:
                break
        
        cap.release()
        cv2.destroyAllWindows()
        messagebox.showinfo('Result', 'Generating Dataset Completed !!!')

# GUI setup
window = tk.Tk()
window.title("Face Recognition System")

# Configuring the grid for dynamic resizing
window.grid_columnconfigure(0, weight=1)
window.grid_columnconfigure(1, weight=1)
window.grid_columnconfigure(2, weight=1)

l1 = tk.Label(window, text="Name", font=("Trebuchet MS", 15))
l1.grid(column=0, row=0, sticky="w", padx=5, pady=5)
t1 = tk.Entry(window, width=55, bd=6)
t1.grid(column=1, row=0, columnspan=2, padx=5, pady=5)

l2 = tk.Label(window, text="Age", font=("Trebuchet MS", 15))
l2.grid(column=0, row=1, sticky="w", padx=5, pady=5)
t2 = tk.Entry(window, width=55, bd=6)
t2.grid(column=1, row=1, columnspan=2, padx=5, pady=5)

l3 = tk.Label(window, text="Address", font=("Trebuchet MS", 15))
l3.grid(column=0, row=2, sticky="w", padx=5, pady=5)
t3 = tk.Entry(window, width=55, bd=6)
t3.grid(column=1, row=2, columnspan=2, padx=5, pady=5)

l4 = tk.Label(window, text="E-mail", font=("Trebuchet MS", 15))
l4.grid(column=0, row=3, sticky="w", padx=5, pady=5)
t4 = tk.Entry(window, width=55, bd=6)
t4.grid(column=1, row=3, columnspan=2, padx=5, pady=5)

l5 = tk.Label(window, text="Contact", font=("Trebuchet MS", 15))
l5.grid(column=0, row=4, sticky="w", padx=5, pady=5)
t5 = tk.Entry(window, width=55, bd=6)
t5.grid(column=1, row=4, columnspan=2, padx=5, pady=5)

b1 = tk.Button(window, text="Training", font=("Trebuchet MS", 10), bg='orange', fg='red', command=train_classifier)
b1.grid(column=0, row=5, padx=5, pady=5)

b2 = tk.Button(window, text="Detect the face", font=("Trebuchet MS", 10), bg='green', fg='white', command=detect_face)
b2.grid(column=1, row=5, padx=5, pady=5)

b3 = tk.Button(window, text="Generate Dataset", font=("Trebuchet MS", 10), bg='pink', fg='black', command=generate_dataset)
b3.grid(column=2, row=5, padx=5, pady=5)

window.geometry("700x300")
window.minsize(700, 300)

window.mainloop()
